## Exercício: Usando LSTMs para Classificar o Conjunto de Dados 20 Newsgroups
O conjunto de dados 20 Newsgroups é um problema de classificação bem conhecido. O objetivo é classificar de qual grupo de notícias veio uma postagem específica. Os 20 grupos possíveis são:

`comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale	
talk.politics.misc
talk.politics.guns
talk.politics.mideast	
talk.religion.misc
alt.atheism
soc.religion.christian`

Como você pode ver, alguns pares de grupos podem ser bastante semelhantes, enquanto outros são muito diferentes.

Os dados são fornecidos como um conjunto de treinamento designado de tamanho 11314 e conjunto de teste de tamanho 7532. As 20 categorias são representadas em proporções aproximadamente iguais, portanto a precisão de base é cerca de 5%.

Para começar, revise o código abaixo. Isso irá guiá-lo pelos conceitos básicos de carregar os dados do 20 newsgroups, carregar os dados do GloVe, construir a matriz de embeddings de palavras e construir o modelo LSTM.

Depois de construirmos o primeiro modelo LSTM, será sua vez de construir um e experimentar com os parâmetros.

In [ ]:
# Preliminares

from __future__ import absolute_import, division, print_function  # Compatibilidade Python 2/3

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import tensorflow.keras as keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from sklearn.datasets import fetch_20newsgroups

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_features = 20000
seq_length = 30  # Comprimento das sequências de palavras
batch_size = 32


In [ ]:
# Baixar os dados do 20 newsgroups - já existe um conjunto designado de "train" e "test"

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')


In [ ]:
len(newsgroups_train.data), len(newsgroups_test.data)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(newsgroups_train.data)

In [ ]:
sequences_train = tokenizer.texts_to_sequences(newsgroups_train.data)
sequences_test = tokenizer.texts_to_sequences(newsgroups_test.data)

In [ ]:
word_index = tokenizer.word_index
print('Encontrados %s tokens únicos.' % len(word_index))


In [ ]:
x_train = pad_sequences(sequences_train, maxlen=seq_length)
x_test = pad_sequences(sequences_test, maxlen=seq_length)


In [ ]:
x_train

In [ ]:
y_train = keras.utils.to_categorical(np.asarray(newsgroups_train.target))
y_test = keras.utils.to_categorical(np.asarray(newsgroups_test.target))

Usaremos os vetores de palavras pré-treinados GloVe. Se você ainda não o fez, por favor baixe-os usando este link:
(NOTA: isso iniciará o download de um arquivo de 822MB)

http://nlp.stanford.edu/data/glove.6B.zip

Em seguida, descompacte o arquivo e preencha seu caminho local para o arquivo na célula de código abaixo.

Usaremos o arquivo `glove.6B.100d.txt`

In [ ]:
embeddings_index = {}
f = open('/Users/lucenator/Work/Data/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Encontrados %s vetores de palavras.' % len(embeddings_index))

Vamos apenas observar um embedding de palavra

In [ ]:
dog_vec = embeddings_index['dog']
dog_vec

In [ ]:
## Isso cria uma matriz onde a i-ésima linha fornece o embedding de palavra para a palavra representada pelo inteiro i.
## Essencialmente, esses serão os "pesos" para a Camada de Embedding
## Em vez de aprender os pesos, usaremos esses e "congelaremos" a camada

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # palavras não encontradas no índice de embeddings serão todas zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

## Camada LSTM
`keras.layers.recurrent.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Estrutura similar à camada `SimpleRNN`
- `units` define a dimensão do estado recorrente
- `recurrent_...` refere-se aos aspectos do estado recorrente do LSTM
- `kernel_...` refere-se às transformações feitas na entrada


In [ ]:
word_dimension = 100  # Esta é a dimensão das palavras que estamos usando do GloVe
model = Sequential([
    Embedding(len(word_index) + 1,
              word_dimension,  
              weights=[embedding_matrix],  # Definimos os pesos como os vetores de palavras do GloVe
              input_length=seq_length,
              trainable=False),  # Ao definir trainable como False, "congelamos" os embeddings de palavras.
    LSTM(30, dropout=0.2, recurrent_dropout=0.2),
    Dense(20, activation='softmax')
])

model.summary()

In [ ]:
rmsprop = keras.optimizers.RMSprop(learning_rate=.002)

model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

## Exercício
### Sua Vez
- Construa uma rede neural com um SimpleRNN em vez de um LSTM (com outras dimensões e parâmetros iguais). Como o desempenho se compara?
- Use o LSTM acima sem os vetores de palavras pré-treinados (inicialize aleatoriamente os pesos e deixe-os serem aprendidos durante o processo de treinamento). Como o desempenho se compara?
- Experimente diferentes comprimentos de sequência e dimensões para o estado oculto do LSTM. Você consegue melhorar o modelo?


In [ ]:
# Por favor, forneça seu código aqui